In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [2]:
!pip install tqdm

In [3]:
import json
with open('/Users/navnoorsingh/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [4]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
print(type(results))
results.keys()


<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [6]:
# set our API call parameters 
LOCATION = 'NY,NY'
TERM = 'Pizza'

In [7]:
search_results = yelp_api.search_query(location = 'NY,NY',
                                      term = 'Pizza')

print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [8]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"
JSON_FILE


'Data/results_in_progress_NY_pizza.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results


15400

In [13]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page


20

In [14]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


770

In [15]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/770 [00:00<?, ?it/s]

In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)


  0%|          | 0/770 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [19]:
#deleting file and confirming that it is deleted.
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)


False

In [26]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == True:
        if delete_if_exists==True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
    else:
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        folder = os.path.dirname(JSON_FILE)
        if len(folder)>0:
            os.makedirs(folder,exist_ok=True)
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  


In [27]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


780

In [28]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/780 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [29]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/I4gm7i...,False,https://www.yelp.com/biz/prince-street-pizza-n...,4751,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,2209.311618
1,ysqgdbSrezXgVwER2kQWKA,julianas-brooklyn-3,Juliana's,https://s3-media2.fl.yelpcdn.com/bphoto/NVoLFl...,False,https://www.yelp.com/biz/julianas-brooklyn-3?a...,2601,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.70274718768062, 'longitude': -...",[delivery],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,1289.857286
2,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2904,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,2268.491950
3,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/njQTdI...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,867,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}",[delivery],$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,1868.295512
4,KEEXuHCYhsQJ-zpJhu98bA,joes-pizza-new-york-148,Joe's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/VkqlwR...,False,https://www.yelp.com/biz/joes-pizza-new-york-1...,1766,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7547, 'longitude': -73.98696}","[pickup, delivery]",$,"{'address1': '1435 Broadway', 'address2': '', ...",+16465594878,(646) 559-4878,5345.192429


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,sYRhc9liDE2tzMyS-pVwhg,sunny-john-pizzeria-brooklyn,Sunny John Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/9ZMSG4...,False,https://www.yelp.com/biz/sunny-john-pizzeria-b...,41,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.6190419, 'longitude': -73.9226...","[pickup, delivery]",$,"{'address1': '5408 Ave N', 'address2': '', 'ad...",+17182416115,(718) 241-6115,10878.347197
996,2JX46610C2fNEyZgcKgvLw,ipizzany-new-york-5,IPizzaNY,https://s3-media1.fl.yelpcdn.com/bphoto/ZEVyv3...,False,https://www.yelp.com/biz/ipizzany-new-york-5?a...,14,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.81108, 'longitude': -73.95293}","[pickup, delivery]",NaN,"{'address1': '357 W 125th St', 'address2': '',...",+19172658973,(917) 265-8973,11787.369722
997,Yubu48l7he-qrNDSeAngbQ,via-vai-astoria-2,Via Vai,https://s3-media3.fl.yelpcdn.com/bphoto/zauzJy...,False,https://www.yelp.com/biz/via-vai-astoria-2?adj...,242,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.7741502, 'longitude': -73.9123...","[pickup, delivery]",$$,"{'address1': '31-09 23rd Ave', 'address2': '',...",+13476124334,(347) 612-4334,9366.001878
998,p-BHJdQ57eM-WlKTP7B44w,antika-astoria,Antika,https://s3-media4.fl.yelpcdn.com/bphoto/zPNvyS...,False,https://www.yelp.com/biz/antika-astoria?adjust...,477,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 40.76466, 'longitude': -73.91722}","[pickup, delivery]",$$,"{'address1': '36-08 30th Ave', 'address2': Non...",+17185450555,(718) 545-0555,8275.396109
999,qZK3e3kZAuyVlNrFNjBZ3A,enoteca-on-court-brooklyn-2,Enoteca on Court,https://s3-media2.fl.yelpcdn.com/bphoto/OWhR-Q...,False,https://www.yelp.com/biz/enoteca-on-court-broo...,262,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.6821365356445, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '347 Court St', 'address2': '', '...",+17182431000,(718) 243-1000,3097.121419


In [30]:
# check for duplicate results
final_df.duplicated().sum()

TypeError: unhashable type: 'list'

In [31]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()


251

In [32]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)
